---
## Step 1: 必要なライブラリをインストール

In [ ]:
# ライブラリのインストール
!pip install streamlit pandas plotly pyngrok -q

print("✅ インストール完了！")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.1 MB/s eta 0:00:00
✅ インストール完了！


---
## Step 2: 財務データを定義

J-POWERの過去4年分の財務データです。

In [17]:
# =====================================
# data.py - 財務データ
# =====================================

# 財務データ（単位：億円）
financial_data = {
    "period": ["2022.3", "2023.3", "2024.3", "2025.3"],
    "revenue": [10846, 18419, 12579, 13167],           # 売上高
    "operating_profit": [1045, 1838, 1057, 1383],      # 営業利益
    "ordinary_profit": [1089, 1975, 1180, 1430],       # 経常利益
    "net_income": [758, 1501, 880, 950],               # 純利益
    "total_assets": [45200, 47800, 46500, 47000],      # 総資産
    "equity": [15100, 16800, 17000, 17200],            # 自己資本
    "debt": [18500, 17500, 17000, 16500],              # 有利子負債
    "operating_cf": [1800, 2500, 2000, 2200],          # 営業CF
    "investing_cf": [-1200, -1500, -1400, -1600],      # 投資CF
}

# セグメント別データ（2025.3期）
segment_data = {
    "name": ["国内発電", "海外事業", "電力周辺", "送変電他"],
    "profit": [685, 345, 340, 12],
}

# 同業他社データ
peer_data = {
    "company": ["J-POWER", "JERA", "東電HD", "関西電力", "中部電力"],
    "equity_ratio": [36.4, 28.5, 25.1, 25.2, 36.4],
    "roe": [7.2, 12.0, 5.5, 21.8, 17.4],
}

print("✅ 財務データを定義しました")
print(f"   対象期間: {financial_data['period'][0]} 〜 {financial_data['period'][-1]}")

✅ 財務データを定義しました
   対象期間: 2022.3 〜 2025.3


---
## Step 3: KPI計算関数を定義

ROE、自己資本比率などの重要指標を計算します。

In [18]:
# =====================================
# calculator.py - KPI計算
# =====================================

def calculate_roe(net_income, equity):
    """ROE（自己資本利益率）= 純利益 ÷ 自己資本 × 100"""
    return round((net_income / equity) * 100, 1)

def calculate_equity_ratio(equity, total_assets):
    """自己資本比率 = 自己資本 ÷ 総資産 × 100"""
    return round((equity / total_assets) * 100, 1)

def calculate_operating_margin(operating_profit, revenue):
    """営業利益率 = 営業利益 ÷ 売上高 × 100"""
    return round((operating_profit / revenue) * 100, 1)

def calculate_fcf(operating_cf, investing_cf):
    """FCF = 営業CF + 投資CF"""
    return operating_cf + investing_cf

def calculate_yoy_change(current, previous):
    """前年比 = (今年 - 去年) ÷ 去年 × 100"""
    if previous == 0:
        return 0
    return round(((current - previous) / previous) * 100, 1)

def calculate_all_kpis(data):
    """すべてのKPIをまとめて計算"""
    latest = -1
    previous = -2

    return {
        "period": data["period"][latest],
        "revenue": data["revenue"][latest],
        "operating_profit": data["operating_profit"][latest],
        "net_income": data["net_income"][latest],
        "roe": calculate_roe(data["net_income"][latest], data["equity"][latest]),
        "equity_ratio": calculate_equity_ratio(data["equity"][latest], data["total_assets"][latest]),
        "operating_margin": calculate_operating_margin(data["operating_profit"][latest], data["revenue"][latest]),
        "fcf": calculate_fcf(data["operating_cf"][latest], data["investing_cf"][latest]),
        "revenue_yoy": calculate_yoy_change(data["revenue"][latest], data["revenue"][previous]),
        "op_profit_yoy": calculate_yoy_change(data["operating_profit"][latest], data["operating_profit"][previous]),
    }

# 計算実行
kpis = calculate_all_kpis(financial_data)

print("✅ KPI計算完了")
print("")
print("=== 計算結果 ===")
for key, value in kpis.items():
    print(f"  {key}: {value}")

✅ KPI計算完了

=== 計算結果 ===
  period: 2025.3
  revenue: 13167
  operating_profit: 1383
  net_income: 950
  roe: 5.5
  equity_ratio: 36.6
  operating_margin: 10.5
  fcf: 600
  revenue_yoy: 4.7
  op_profit_yoy: 30.8


---
## Step 4: AIコメント生成関数を定義

KPIデータから自動でコメントを生成します（API不要版）。

In [19]:
# =====================================
# ai_comment.py - コメント生成
# =====================================

def generate_comment(kpi_data):
    """ルールベースでコメントを生成（API不要）"""
    comments = []

    # 売上高
    if kpi_data['revenue_yoy'] > 0:
        comments.append(
            f"売上高は{kpi_data['revenue']:,}億円で、"
            f"前年比+{kpi_data['revenue_yoy']}%と増収となりました。"
        )
    else:
        comments.append(
            f"売上高は{kpi_data['revenue']:,}億円で、"
            f"前年比{kpi_data['revenue_yoy']}%と減収となりました。"
        )

    # 営業利益
    if kpi_data['op_profit_yoy'] > 10:
        comments.append(
            f"営業利益は{kpi_data['operating_profit']:,}億円"
            f"（前年比+{kpi_data['op_profit_yoy']}%）と大幅増益です。"
        )
    elif kpi_data['op_profit_yoy'] > 0:
        comments.append(
            f"営業利益は{kpi_data['operating_profit']:,}億円"
            f"（前年比+{kpi_data['op_profit_yoy']}%）と増益基調です。"
        )
    else:
        comments.append(
            f"営業利益は{kpi_data['operating_profit']:,}億円"
            f"（前年比{kpi_data['op_profit_yoy']}%）と減益となりました。"
        )

    # ROE
    if kpi_data['roe'] >= 8:
        comments.append(f"ROEは{kpi_data['roe']}%と目標水準（8%）を達成しています。")
    elif kpi_data['roe'] >= 5:
        comments.append(f"ROEは{kpi_data['roe']}%で、目標（8%）に向けた改善が期待されます。")
    else:
        comments.append(f"ROEは{kpi_data['roe']}%と低水準であり、資本効率の改善が課題です。")

    # 自己資本比率
    if kpi_data['equity_ratio'] >= 30:
        comments.append(
            f"自己資本比率は{kpi_data['equity_ratio']}%と財務健全性は高い水準を維持しています。"
        )
    else:
        comments.append(
            f"自己資本比率は{kpi_data['equity_ratio']}%であり、財務基盤の強化が望まれます。"
        )

    return "\n".join(comments)

# コメント生成実行
comment = generate_comment(kpis)

print("✅ コメント生成完了")
print("")
print("=== 生成されたコメント ===")
print(comment)

✅ コメント生成完了

=== 生成されたコメント ===
売上高は13,167億円で、前年比+4.7%と増収となりました。
営業利益は1,383億円（前年比+30.8%）と大幅増益です。
ROEは5.5%で、目標（8%）に向けた改善が期待されます。
自己資本比率は36.6%と財務健全性は高い水準を維持しています。


---
## Step 5: Streamlitダッシュボードを起動

* ngrokを使用。

In [20]:
from pyngrok import ngrok
import time

# ngrokの認証
ngrok.set_auth_token("3836JqSEZPNejyYpvZiYZk8bejp_5xyHJVzB6fr9bqm2mQjEB")

# Streamlitをバックグラウンドで起動
!nohup streamlit run app.py --server.port 8501 --server.headless true > /dev/null 2>&1 &

# 起動を待つ
print("⏳ Streamlitを起動中...")
time.sleep(5)

# ngrokでトンネルを作成
public_url = ngrok.connect(8501)

print("")
print("=" * 50)
print("🎉 ダッシュボードが起動しました！")
print("=" * 50)
print("")
print("👇 以下のURLをクリックしてください 👇")
print("")
print(f"🔗 {public_url}")
print("")
print("=" * 50)

⏳ Streamlitを起動中...

🎉 ダッシュボードが起動しました！

👇 以下のURLをクリックしてください 👇

🔗 NgrokTunnel: "https://plagiocephalic-munificently-tara.ngrok-free.dev" -> "http://localhost:8501"

